In [1]:
import os
import math
import pandas as pd
import numpy as np


os.environ['QT_DEVICE_PIXEL_RATIO'] = ''  # disable annoying warnings on plotting


def get_datafiles_info(prepath="ants/"):
    indoor = [("indoor", prepath+'Ant_dataset_noimg/IndoorDataset/'+p +
               "/gt/gt.txt") for p in os.listdir(prepath+'Ant_dataset_noimg/IndoorDataset')]
    outdoor = [("outdoor", prepath+'Ant_dataset_noimg/OutdoorDataset/'+p +
                "/gt/gt.txt") for p in os.listdir(prepath+'Ant_dataset_noimg/OutdoorDataset')]

    data_locations = pd.DataFrame(indoor+outdoor, columns=['recording_location', 'file'])
    return data_locations


def preprocess_antcolony_data(datafile):
    ants = pd.read_csv(datafile, names=['frameno', 'ant_id',
                                        'bbleft', 'bbtop', 'bbwidth', 'bbheight', 'confidence'])

    # there are multiple rows per frame, each row represents one ant in one frame, so there are multiple ants per frame
    # the ant_id is the unique identifier for each ant
    # ants have bounding boxes (boxes in which they are detected), so bbleft and bbtop can be used as x and y coordinates
    ants['x'] = ants['bbleft'] - ants['bbwidth'] / 2
    ants['y'] = ants['bbtop'] - ants['bbheight'] / 2

    ants.drop(columns=['bbleft', 'bbtop', 'bbwidth', 'bbheight', 'confidence'], inplace=True)

    # use groupby to make sure the shifted postions do not "leak" between different ants
    ants['next_x'] = ants.groupby('ant_id')['x'].shift(-1)
    ants['next_y'] = ants.groupby('ant_id')['y'].shift(-1)
    ants['speed'] = ((ants['next_y'] - ants['y'])**2 + (ants['next_x'] - ants['x'])**2)**0.5
    heading = (ants['next_y'] - ants['y']) / (ants['next_x'] - ants['x'])

    # convert heading to angle to ensure remaining in cos domain
    ants['angle'] = heading.apply(lambda x: math.atan(x) if x != 0 else 0)
    ants['cos(angle)'] = ants['angle'].apply(lambda x: math.cos(x))
    return ants

In [16]:
get_datafiles_info()

,recording_location,file
0,indoor,ants/Ant_dataset_noimg/IndoorDataset/Seq0001Ob...
1,indoor,ants/Ant_dataset_noimg/IndoorDataset/Seq0002Ob...
2,indoor,ants/Ant_dataset_noimg/IndoorDataset/Seq0003Ob...
3,indoor,ants/Ant_dataset_noimg/IndoorDataset/Seq0004Ob...
4,indoor,ants/Ant_dataset_noimg/IndoorDataset/Seq0005Ob...
5,outdoor,ants/Ant_dataset_noimg/OutdoorDataset/Seq0006O...
6,outdoor,ants/Ant_dataset_noimg/OutdoorDataset/Seq0007O...
7,outdoor,ants/Ant_dataset_noimg/OutdoorDataset/Seq0008O...
8,outdoor,ants/Ant_dataset_noimg/OutdoorDataset/Seq0009O...
9,outdoor,ants/Ant_dataset_noimg/OutdoorDataset/Seq0010O...


In [21]:
data_info = get_datafiles_info()
selected_files = data_info['file'].iloc[0:10]

for i, file_path in enumerate(selected_files):
    print(file_path)
    ants = preprocess_antcolony_data(file_path)

    # Generate a unique filename for each file
    # This assumes files have a readable structure and you can name them sequentially
    # name which are indoor and which are outdoor

    output_filename = f'ants_seq{i+1}.csv'  
    ants.to_csv(output_filename)

    print(ants)

    
    print(f"Data written to {output_filename}")

ants/Ant_dataset_noimg/IndoorDataset/Seq0001Object10Image94/gt/gt.txt
      frameno  ant_id         x         y   next_x    next_y      speed  \
0           1      11   693.603    1.5630  674.258    8.8174  20.660478   
1           1      12   671.840  639.9510  674.258  644.7870   5.406812   
2           1      13   758.893  777.7850  746.802  777.7850  12.091000   
3           1      14   780.656  695.5680  778.238  690.7320   5.406812   
4           1      15   843.528  303.8300  845.946  296.5760   7.646387   
...       ...     ...       ...       ...      ...       ...        ...   
3505      351      16  1087.760  226.4500      NaN       NaN        NaN   
3506      351      17   855.618  -22.6184      NaN       NaN        NaN   
3507      351      18   962.020  199.8500      NaN       NaN        NaN   
3508      351      19  1128.870  378.7920      NaN       NaN        NaN   
3509      351      20   691.185   74.1070      NaN       NaN        NaN   

         angle  cos(angle)  


In [34]:
import pandas as pd
from pyrqa.settings import Settings
from pyrqa.neighbourhood import FixedRadius
from pyrqa.time_series import TimeSeries
from pyrqa.computation import RQAComputation

def load_and_preprocess_data(file_name):
    data = pd.read_csv(file_name)
    # Assuming 'x' and 'y' are the coordinates we're interested in
    return data['x'].values, data['y'].values  # Separate the dimensions for individual processing

def perform_RQA(time_series_data, radius=0.5, embedding_dimension=2, time_delay=1):
    time_series = TimeSeries(time_series_data, embedding_dimension=embedding_dimension, time_delay=time_delay)
    settings = Settings(time_series, neighbourhood=FixedRadius(radius))
    result = RQAComputation.create(settings).run()
    return result

# Iterate over your CSV files
for i in range(1, 11):
    file_name = fr'C:\Users\Barto\Desktop\Complex Systems\Complex_Systems_2024\ants_seq{i}.csv'
    x_data, y_data = load_and_preprocess_data(file_name)

    # Analyze each dimension separately
    print(f'Results for {file_name}:')
    for dimension, data in zip(['x', 'y'], [x_data, y_data]):
        rqa_result = perform_RQA(data)
        print(f'Dimension {dimension}:')
        print(rqa_result)



Results for C:\Users\Barto\Desktop\Complex Systems\Complex_Systems_2024\ants_seq1.csv:


c:\Users\Barto\AppData\Local\Programs\Python\Python310\lib\site-packages\pyopencl\cache.py:417: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  prg.build(options_bytes, [devices[i] for i in to_be_built_indices])
c:\Users\Barto\AppData\Local\Programs\Python\Python310\lib\site-packages\pyopencl\cache.py:495: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  _create_built_program_from_source_cached(


Dimension x:
RQA Result:

Minimum diagonal line length (L_min): 2
Minimum vertical line length (V_min): 2
Minimum white vertical line length (W_min): 2

Recurrence rate (RR): 0.000330
Determinism (DET): 0.171429
Average diagonal line length (L): 2.086957
Longest diagonal line length (L_max): 3
Divergence (DIV): 0.333333
Entropy diagonal lines (L_entr): 0.295439
Laminarity (LAM): 0.000000
Trapping time (TT): nan
Longest vertical line length (V_max): 1
Entropy vertical lines (V_entr): 0.000000
Average white vertical line length (W): 1625.380959
Longest white vertical line length (W_max): 3508
Longest white vertical line length inverse (W_div): 0.000285
Entropy white vertical lines (W_entr): 7.965283

Ratio determinism / recurrence rate (DET/RR): 518.754949
Ratio laminarity / determinism (LAM/DET): 0.000000

Dimension y:
RQA Result:

Minimum diagonal line length (L_min): 2
Minimum vertical line length (V_min): 2
Minimum white vertical line length (W_min): 2

Recurrence rate (RR): 0.000390

c:\Users\Barto\AppData\Local\Programs\Python\Python310\lib\site-packages\pyopencl\cache.py:495: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  _create_built_program_from_source_cached(


Dimension y:
RQA Result:

Minimum diagonal line length (L_min): 2
Minimum vertical line length (V_min): 2
Minimum white vertical line length (W_min): 2

Recurrence rate (RR): 0.000452
Determinism (DET): 0.157282
Average diagonal line length (L): 2.160000
Longest diagonal line length (L_max): 4
Divergence (DIV): 0.250000
Entropy diagonal lines (L_entr): 0.459395
Laminarity (LAM): 0.000000
Trapping time (TT): nan
Longest vertical line length (V_max): 1
Entropy vertical lines (V_entr): 0.000000
Average white vertical line length (W): 1357.096262
Longest white vertical line length (W_max): 3508
Longest white vertical line length inverse (W_div): 0.000285
Entropy white vertical lines (W_entr): 7.471664

Ratio determinism / recurrence rate (DET/RR): 347.750136
Ratio laminarity / determinism (LAM/DET): 0.000000

Results for C:\Users\Barto\Desktop\Complex Systems\Complex_Systems_2024\ants_seq3.csv:
Dimension x:
RQA Result:

Minimum diagonal line length (L_min): 2
Minimum vertical line length (

...

Dimension x:
RQA Result:

Minimum diagonal line length (L_min): 2
Minimum vertical line length (V_min): 2
Minimum white vertical line length (W_min): 2

Recurrence rate (RR): 0.000112
Determinism (DET): 0.245330
Average diagonal line length (L): 2.238636
Longest diagonal line length (L_max): 5
Divergence (DIV): 0.200000
Entropy diagonal lines (L_entr): 0.601943
Laminarity (LAM): 0.000000
Trapping time (TT): nan
Longest vertical line length (V_max): 1
Entropy vertical lines (V_entr): 0.000000
Average white vertical line length (W): 4767.416606
Longest white vertical line length (W_max): 10278
Longest white vertical line length inverse (W_div): 0.000097
Entropy white vertical lines (W_entr): 9.047608

Ratio determinism / recurrence rate (DET/RR): 2180.987753
Ratio laminarity / determinism (LAM/DET): 0.000000

Dimension y:
RQA Result:

Minimum diagonal line length (L_min): 2
Minimum vertical line length (V_min): 2
Minimum white vertical line length (W_min): 2

Recurrence rate (RR): 0.0001

c:\Users\Barto\AppData\Local\Programs\Python\Python310\lib\site-packages\pyopencl\cache.py:495: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  _create_built_program_from_source_cached(


Dimension x:
RQA Result:

Minimum diagonal line length (L_min): 2
Minimum vertical line length (V_min): 2
Minimum white vertical line length (W_min): 2

Recurrence rate (RR): 0.000056
Determinism (DET): 0.430903
Average diagonal line length (L): 2.570342
Longest diagonal line length (L_max): 12
Divergence (DIV): 0.083333
Entropy diagonal lines (L_entr): 1.020589
Laminarity (LAM): 0.000000
Trapping time (TT): nan
Longest vertical line length (V_max): 1
Entropy vertical lines (V_entr): 0.000000
Average white vertical line length (W): 10889.308358
Longest white vertical line length (W_max): 27900
Longest white vertical line length inverse (W_div): 0.000036
Entropy white vertical lines (W_entr): 9.158525

Ratio determinism / recurrence rate (DET/RR): 7695.586919
Ratio laminarity / determinism (LAM/DET): 0.000000

Dimension y:
RQA Result:

Minimum diagonal line length (L_min): 2
Minimum vertical line length (V_min): 2
Minimum white vertical line length (W_min): 2

Recurrence rate (RR): 0.00

c:\Users\Barto\AppData\Local\Programs\Python\Python310\lib\site-packages\pyopencl\cache.py:495: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  _create_built_program_from_source_cached(


Dimension x:
RQA Result:

Minimum diagonal line length (L_min): 2
Minimum vertical line length (V_min): 2
Minimum white vertical line length (W_min): 2

Recurrence rate (RR): 0.000061
Determinism (DET): 0.082664
Average diagonal line length (L): 2.126667
Longest diagonal line length (L_max): 4
Divergence (DIV): 0.250000
Entropy diagonal lines (L_entr): 0.354573
Laminarity (LAM): 0.003091
Trapping time (TT): 2.000000
Longest vertical line length (V_max): 2
Entropy vertical lines (V_entr): 0.000000
Average white vertical line length (W): 9396.485892
Longest white vertical line length (W_max): 22042
Longest white vertical line length inverse (W_div): 0.000045
Entropy white vertical lines (W_entr): 9.560742

Ratio determinism / recurrence rate (DET/RR): 1349.614656
Ratio laminarity / determinism (LAM/DET): 0.037396

Dimension y:
RQA Result:

Minimum diagonal line length (L_min): 2
Minimum vertical line length (V_min): 2
Minimum white vertical line length (W_min): 2

Recurrence rate (RR): 0

In [ ]:
# analyze on an ant level whether an ants position recurrs with iteself (potentially explaining behavior of a single ant)